In [103]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from copy import deepcopy
from label_manipulation import median_filter_labels,fill_closest
import tensorflow as tf

from SignalRecognition.cwt_learner.wavelet_feature_engineering import CWT_learner
from signal_database.labelSelectedData import SignalDB,SignalBundle
import matplotlib.pyplot as plt
from plot_generator import plotResult_colorbars

In [2]:
sdb = SignalDB('grab_objects', path='./')
training_data_ = sdb.get_labeleddata()

In [3]:
%matplotlib

Using matplotlib backend: TkAgg


In [4]:
plt.figure()
for ii,example in enumerate(training_data_):
    plt.subplot(len(training_data_)*2,1,ii*2 + 1)
    for channel in example.signal_bundle.signals:
        plt.plot(channel)
    plt.subplot(len(training_data_)*2,1,ii*2 + 2)
    labels = example.labels
    plotResult_colorbars(labels, range(len(labels)))
plt.show()

In [5]:
all_possible_labels = []
for example in training_data_:
        all_possible_labels = all_possible_labels + example.labels
print set(all_possible_labels)



set(['', 'freespace', 'hold_foamblock', 'grab_hex', 'hand_on', 'grab_cup', 'release_foamblock', 'release_cup', 'hold_cup', 'grab_foamblock', 'still', 'hold_hex', 'release_hex', 'hand_off'])


In [6]:
training_data_relabeled = deepcopy(training_data_)


In [7]:
# label map 
label_map = {
    "freespace" : "freespace",
    "grab_hex" : "grab",
    "grab_cup" : "grab",
    "grab_foamblock" : "grab",
    "release_foamblock" : "release",
    "release_cup" : "release",
    "release_hex" : "release",
    "hold_cup" : "hold",
    "hold_hex" : "hold",
    "hold_foamblock" : "hold",
    "hand_on" : "hand_on",
    "hand_off" : "hand_off",
    "still" : "still",
    "" : ""
}



for example in training_data_relabeled:
        example.labels = [label_map[label] for label in example.labels]


In [8]:
cwt_learn_training = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in training_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn_training.add_training_data(signals,labels)

(trainX,trainY) = cwt_learn_training.get_examples_with_weights(cwt_learn_training.training_data_sets,with_window = "Off",
                                                         signal_indices = cwt_learn_training.signal_indices
                                                         ,wavelet = cwt_learn_training.wavelet)

all_labels = []
for example in training_data_relabeled:
        all_labels = example.labels + all_labels
label_types = list(set(all_labels))
print label_types


['', 'freespace', 'hold', 'hand_off', 'hand_on', 'release', 'grab', 'still']


In [9]:
cwt_learn_test = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in testing_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn_test.add_training_data(signals,labels)
    
(testX,testY) = cwt_learn_test.get_examples_with_weights(cwt_learn_test.training_data_sets,with_window = "Off",
                                                         signal_indices = cwt_learn_test.signal_indices
                                                         ,wavelet = cwt_learn_test.wavelet)


In [10]:
trainX = np.array(trainX)
trainY = np.array(trainY)
testX = np.array(testX)
testY = np.array(testY)

In [11]:
from label_manipulation import fill_closest
trainY_filled = fill_closest(trainY)

In [12]:
import pandas as pd
from scipy.interpolate import interp1d

In [13]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [14]:
Y_one_hot = pd.get_dummies(trainY_filled)
print np.shape(trainX)[1], np.shape(Y_one_hot)
label_names_df = list(Y_one_hot.columns.values)
np.array(Y_one_hot.idxmax(axis=1))

80 (367130, 7)


array(['still', 'still', 'still', ..., 'still', 'still', 'still'],
      dtype=object)

In [15]:
def baseline_model():
    model = Sequential()
    model.add(Dense(10,input_dim=np.shape(trainX)[1],activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(np.shape(Y_one_hot)[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# estimator = KerasClassifier(build_fn=baseline_model, epochs=1, batch_size=1000, verbose=1)

In [16]:
trainY_one_hot = np.array(Y_one_hot)
# train model against trainX and trainY_one_hot

In [40]:
model = baseline_model()
model.fit(trainX,Y_one_hot,epochs = 10, batch_size=20)

Epoch 1/10


    20/367130 [..............................] - ETA: 1:22:59 - loss: 10.9367 - acc: 0.0000e+00

   480/367130 [..............................] - ETA: 4:07 - loss: 6.6670 - acc: 0.0417        

   960/367130 [..............................] - ETA: 2:22 - loss: 4.4135 - acc: 0.1115

  1780/367130 [..............................] - ETA: 1:27 - loss: 3.1893 - acc: 0.2039

...

362620/367130 [============================>.] - ETA: 0s - loss: 0.1278 - acc: 0.9517

363740/367130 [============================>.] - ETA: 0s - loss: 0.1279 - acc: 0.9517

364880/367130 [============================>.] - ETA: 0s - loss: 0.1279 - acc: 0.9516

366020/367130 [============================>.] - ETA: 0s - loss: 0.1278 - acc: 0.9517

367130/367130 [==============================] - 17s 46us/step - loss: 0.1279 - acc: 0.9516


In [44]:
# computing sample weights
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight('balanced',trainY_filled)

In [47]:
model_sample_weights = baseline_model()
model_sample_weights.fit(trainX,Y_one_hot,epochs = 10, batch_size=20,sample_weight=sample_weights)

Epoch 1/10


    20/367130 [..............................] - ETA: 1:32:59 - loss: 4.1078 - acc: 0.1000

   580/367130 [..............................] - ETA: 3:44 - loss: 2.8085 - acc: 0.1190   

  1180/367130 [..............................] - ETA: 2:05 - loss: 2.4367 - acc: 0.1195

  1600/367130 [..............................] - ETA: 1:44 - loss: 2.3178 - acc: 0.1325

...

363780/367130 [============================>.] - ETA: 0s - loss: 0.1347 - acc: 0.9373

364900/367130 [============================>.] - ETA: 0s - loss: 0.1347 - acc: 0.9373

366000/367130 [============================>.] - ETA: 0s - loss: 0.1346 - acc: 0.9374

367020/367130 [============================>.] - ETA: 0s - loss: 0.1345 - acc: 0.9374

367130/367130 [==============================] - 17s 46us/step - loss: 0.1346 - acc: 0.9374


In [195]:
# plt.subplot(2,1,1)
# plotResult_colorbars(out_list, range(len(out_list)),labelNames=label_types)
# plt.subplot(2,1,2)
# plotResult_colorbars(trainY, range(len(trainY)),labelNames=label_types)

In [ ]:
test_one_hot_predict = model_sample_weights.predict(testX)

test_one_hot_predict = (test_one_hot_predict > 0.9).astype(int)
test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)
test_one_hot_predict_series_ = list(test_one_hot_predict_df.idxmax(axis = 1))
test_one_hot_predict_series = median_filter_labels(test_one_hot_predict_series_,window=1001)

In [52]:
ax = plt.subplot(2,1,1)

plotResult_colorbars(test_one_hot_predict_series, range(len(test_one_hot_predict_series)),
                     labelNames=label_types,ax=ax)
ax.set_xlim([0,len(test_one_hot_predict_series)])
ax = plt.subplot(2,1,2)
plotResult_colorbars(fill_closest(testY), range(len(testY)),labelNames=label_types,ax=ax)
ax.set_xlim([0,len(test_one_hot_predict_series)])


(0, 198932)

In [55]:
## trying LSTM
# training set: trainX, Y_one_hot -> the order is correct

In [66]:
from keras.layers import LSTM

In [299]:

data_dim = 80
timesteps = 100
batch_size = 100
num_classes = len(label_names_df)
# expected input data shape: (batch_size, timesteps, data_dim)
model_LSTM = Sequential()
model_LSTM.add(LSTM(50, return_sequences=True,
               input_shape=(timesteps, data_dim),batch_size = batch_size
                    ,stateful=True))  # returns a sequence of vectors of dimension 32
model_LSTM.add(LSTM(50,stateful=True))  # return a single vector of dimension 32
model_LSTM.add(Dense(50, activation='relu'))
# model_LSTM.add(Dense(10, activation='relu'))
model_LSTM.add(Dense(num_classes, activation='softmax'))
model_LSTM.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [67]:
def LSTM_model(data_dim,timesteps,batch_size,num_classes,stateful = True,
               sample_weight_mode = None):
    # expected input data shape: (batch_size, timesteps, data_dim)
    model_LSTM = Sequential()
    model_LSTM.add(LSTM(50, return_sequences=True,
                   input_shape=(timesteps, data_dim),batch_size = batch_size
                        ,stateful=stateful))  # returns a sequence of vectors of dimension 32
    model_LSTM.add(LSTM(50,stateful=stateful))  # return a single vector of dimension 32
    model_LSTM.add(Dense(50, activation='relu'))
    # model_LSTM.add(Dense(10, activation='relu'))
    model_LSTM.add(Dense(num_classes, activation='softmax'))
    model_LSTM.compile( loss='categorical_crossentropy',
                        optimizer='rmsprop',
                        metrics=['accuracy'],
                        sample_weight_mode=sample_weight_mode)
    return model_LSTM
    
    
data_dim = 80
timesteps = 100
batch_size = 100
num_classes = len(label_names_df)

model_LSTM = LSTM_model(data_dim,timesteps,batch_size,num_classes)
model_LSTM_nostateful = LSTM_model(data_dim,timesteps,batch_size,num_classes,stateful=False)
    

In [68]:
from keras.utils import Sequence
import numpy as np


class SeqGen(Sequence):
    def __init__(self, x_set, y_set, batch_size, window_size,batch_stride = 1,
                 sample_weights = None):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.window_size = window_size
        self.batch_stride = batch_stride
        self.sw = sample_weights
            

    def __len__(self):
        return len(self.x)//self.batch_stride

    def __getitem__(self, idx):
        batch_x = []
        batch_y = []

        current_x = np.roll(self.x, idx*self.batch_stride, axis=0)
        current_y = np.roll(self.y, idx*self.batch_stride, axis=0)
        
        for ii in range(self.batch_size):
            batch_x.append(np.roll(current_x, ii,axis = 0)[:self.window_size])
            batch_y.append(np.roll(current_y, ii,axis = 0)[:self.window_size][-1])
        
        if self.sw is None:
            return np.array(batch_x), np.array(batch_y)
        
        else:
            batch_sw = []
            current_sw = np.roll(self.sw, idx*self.batch_stride, axis=0)
            for ii in range(self.batch_size):    
                batch_sw.append(np.roll(current_sw, ii,axis = 0)[:self.window_size][-1])

            return np.array(batch_x), np.array(batch_y), np.array(batch_sw)
    


In [357]:
skip = 100
trainX_sub_sampled = trainX[0:-1:skip]
Y_one_hot_sub_sampled = Y_one_hot[0:-1:skip]

from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight('balanced',trainY_filled)
sample_weights_sub_sampled = sample_weights[0:-1:skip]
sg_train = SeqGen(trainX_sub_sampled,Y_one_hot_sub_sampled,100,100,
                  batch_stride=100,sample_weights=sample_weights_sub_sampled)

In [355]:
testY_filled = fill_closest(testY)
Y_test_one_hot = pd.get_dummies(testY_filled)
# label_names_df = list(Y_one_hot.columns.values)
sg_test = SeqGen(testX[0:-1:100],Y_test_one_hot[0:-1:skip],100,100,batch_stride=100)
sg_valid = SeqGen(testX[50:-1:100],Y_test_one_hot[50:-1:skip],100,100,batch_stride=100)

In [345]:
# np.shape(sg_train[0][2])

(100,)

In [367]:
model_LSTM.fit_generator(sg_train,validation_data=sg_valid,
                    epochs=5)

Epoch 1/5
 1/36 [..............................] - ETA: 6s - loss: 0.2294 - acc: 0.8500

 2/36 [>.............................] - ETA: 5s - loss: 0.8308 - acc: 0.7250

 3/36 [=>............................] - ETA: 5s - loss: 0.7272 - acc: 0.7300

 4/36 [==>...........................] - ETA: 4s - loss: 0.5683 - acc: 0.7825

 5/36 [===>..........................] - ETA: 4s - loss: 0.4666 - acc: 0.8200

...

32/36 [=========================>....] - ETA: 0s - loss: 0.2179 - acc: 0.9203

33/36 [==========================>...] - ETA: 0s - loss: 0.2120 - acc: 0.9224

34/36 [===========================>..] - ETA: 0s - loss: 0.2070 - acc: 0.9235

35/36 [============================>.] - ETA: 0s - loss: 0.2018 - acc: 0.9254

36/36 [==============================] - 4s 98ms/step - loss: 0.1973 - acc: 0.9269 - val_loss: 0.3517 - val_acc: 0.8811


In [365]:
model_LSTM_nostateful.fit_generator(sg_train,validation_data=sg_valid,
                    epochs=5)
    

Epoch 1/5
 1/36 [..............................] - ETA: 6s - loss: 0.3969 - acc: 0.8400

 2/36 [>.............................] - ETA: 5s - loss: 0.2032 - acc: 0.9200

 3/36 [=>............................] - ETA: 4s - loss: 0.1659 - acc: 0.9400

 4/36 [==>...........................] - ETA: 4s - loss: 0.2693 - acc: 0.9100

 5/36 [===>..........................] - ETA: 4s - loss: 0.6057 - acc: 0.8160

...

32/36 [=========================>....] - ETA: 0s - loss: 0.4173 - acc: 0.8403

33/36 [==========================>...] - ETA: 0s - loss: 0.4094 - acc: 0.8436

34/36 [===========================>..] - ETA: 0s - loss: 0.4071 - acc: 0.8447

35/36 [============================>.] - ETA: 0s - loss: 0.4144 - acc: 0.8454

36/36 [==============================] - 4s 100ms/step - loss: 0.4045 - acc: 0.8486 - val_loss: 0.7635 - val_acc: 0.7737


In [368]:
test_one_hot_predict_series = []
test_one_hot_truth_series = []

for ii in range(len(sg_test)):
    test_one_hot_predict = model_LSTM.predict(sg_test[ii][0],batch_size=batch_size)
    test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)
    test_one_hot_predict_series_batch = list(test_one_hot_predict_df.idxmax(axis = 1))
    test_one_hot_predict_series = test_one_hot_predict_series + test_one_hot_predict_series_batch 
    
    test_one_hot_truth = (sg_test[ii][1] > 0.9).astype(int)
    test_one_hot_truth_df = pd.DataFrame(test_one_hot_truth,columns=label_names_df)
    test_one_hot_truth_series_batch = list(test_one_hot_truth_df.idxmax(axis = 1))
    test_one_hot_truth_series = test_one_hot_truth_series + test_one_hot_truth_series_batch

In [363]:
# print test_one_hot_predict_series

In [369]:
# check_id = 20
# 
# test_one_hot_predict = model_LSTM_nostateful.predict(sg_test[check_id][0],batch_size=batch_size)
# test_one_hot_predict = (test_one_hot_predict > 0.9).astype(int)
# test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)
# test_one_hot_predict_series = list(test_one_hot_predict_df.idxmax(axis = 1))
# # test_one_hot_predict_series = median_filter_labels(test_one_hot_predict_series_,window=1001)
# test_one_hot_truth = (sg_test[check_id][1] > 0.9).astype(int)
# test_one_hot_truth_df = pd.DataFrame(test_one_hot_truth,columns=label_names_df)
# test_one_hot_truth_series = list(test_one_hot_truth_df.idxmax(axis = 1))
# test_one_hot_truth_series = median_filter_labels(test_one_hot_truth_series_,window=1001)

ax = plt.subplot(2,1,1)

plotResult_colorbars(test_one_hot_predict_series, range(len(test_one_hot_predict_series)),
                     labelNames=label_types,ax=ax,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_predict_series)])
ax = plt.subplot(2,1,2)

plotResult_colorbars(test_one_hot_truth_series, range(len(test_one_hot_truth_series)),
                     labelNames=label_types,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_truth_series)])


print set(test_one_hot_truth_series)
print set(test_one_hot_predict_series)


set(['freespace', 'still', 'hand_off', 'hand_on', 'release', 'grab', 'hold'])
set(['freespace', 'still', 'hand_off', 'hand_on', 'release', 'grab', 'hold'])


In [145]:
label_types

['', 'freespace', 'hold', 'hand_off', 'hand_on', 'release', 'grab', 'still']

In [108]:
# benchmark
full_train_X = np.array([[]]).reshape(0,len(cwt_learn_training.signal_indices))
full_train_Y = np.array([])
for ii in range(len(cwt_learn_training.training_data_sets)):
    signals = np.transpose(cwt_learn_training.training_data_sets[ii].signal_bundle.signals)
    labels = cwt_learn_training.training_data_sets[ii].labels
    labels = fill_closest(labels)
    full_train_X = np.append(full_train_X,signals,axis = 0)
    full_train_Y = np.append(full_train_Y,labels)
    
    
full_test_X = np.array([[]]).reshape(0,len(cwt_learn_test.signal_indices))
full_test_Y = np.array([])
for ii in range(len(cwt_learn_test.training_data_sets)):
    signals = np.transpose(cwt_learn_test.training_data_sets[ii].signal_bundle.signals)
    labels = cwt_learn_test.training_data_sets[ii].labels
    labels = fill_closest(labels)
    full_test_X = np.append(full_test_X,signals,axis = 0)
    full_test_Y = np.append(full_test_Y,labels)
    
from sklearn.utils.class_weight import compute_sample_weight
sample_weights_train_full = compute_sample_weight('balanced',full_train_Y)
sample_weights_test_full = compute_sample_weight('balanced',full_test_Y)



In [109]:
print np.shape(full_train_X), np.shape(full_train_Y)
print np.shape(full_test_X), np.shape(full_test_Y)

(367130, 4) (367130,)
(198932, 4) (198932,)


In [110]:
full_trainX_sub_sampled = full_train_X[0:-1:100]
full_trainY_sub_sampled = full_train_Y[0:-1:100]
sw_train_full_sub_sampled = sample_weights_train_full[0:-1:100]

full_Y_train_one_hot_sampled = pd.get_dummies(full_trainY_sub_sampled)



full_testX_sub_sampled = full_test_X[0:-1:100]
full_testY_sub_sampled = full_test_Y[0:-1:100]
sw_test_full_sub_sampled = sample_weights_test_full[0:-1:100]

full_Y_test_one_hot_sampled = pd.get_dummies(full_testY_sub_sampled)



full_validationX_sub_sampled = full_test_X[50:-1:100]
full_validationY_sub_sampled = full_test_Y[50:-1:100]
sw_validation_full_sub_sampled = sample_weights_test_full[50:-1:100]

full_Y_validation_one_hot_sampled = pd.get_dummies(full_validationY_sub_sampled)

# trainY_filled = fill_closest(trainY)

In [111]:
data_dim = len(cwt_learn_training.signal_indices)
timesteps = 100
window = 100
batch_stride = 100
batch_size = 100

sg_train_no_wavelet = SeqGen(full_trainX_sub_sampled,full_Y_train_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride,sample_weights=sw_train_full_sub_sampled)

sg_test_no_wavelet = SeqGen(full_testX_sub_sampled,full_Y_test_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride, sample_weights=sw_test_full_sub_sampled)

sg_validation_no_wavelet = SeqGen(full_validationX_sub_sampled,full_Y_validation_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride, sample_weights=sw_validation_full_sub_sampled)

In [112]:

model_LSTM_no_wavelet = LSTM_model(data_dim,timesteps,batch_size,num_classes)

In [117]:
model_LSTM_no_wavelet.fit_generator(sg_train_no_wavelet,validation_data=sg_validation_no_wavelet,
                    epochs=30)


Epoch 1/30
 1/36 [..............................] - ETA: 3s - loss: 0.4725 - acc: 0.9200

 2/36 [>.............................] - ETA: 3s - loss: 0.4549 - acc: 0.7500

 3/36 [=>............................] - ETA: 2s - loss: 0.3707 - acc: 0.8200

 4/36 [==>...........................] - ETA: 3s - loss: 0.3479 - acc: 0.8525

 5/36 [===>..........................] - ETA: 2s - loss: 0.3834 - acc: 0.8300

...

32/36 [=========================>....] - ETA: 0s - loss: 0.3068 - acc: 0.8656

33/36 [==========================>...] - ETA: 0s - loss: 0.3101 - acc: 0.8621

34/36 [===========================>..] - ETA: 0s - loss: 0.3074 - acc: 0.8629

35/36 [============================>.] - ETA: 0s - loss: 0.3061 - acc: 0.8634

36/36 [==============================] - 3s 76ms/step - loss: 0.3020 - acc: 0.8667 - val_loss: 0.3898 - val_acc: 0.8458


In [118]:
test_one_hot_predict_series = []
test_one_hot_truth_series = []

for ii in range(len(sg_test_no_wavelet)):
    test_one_hot_predict = model_LSTM_no_wavelet.predict(sg_test_no_wavelet[ii][0],batch_size=batch_size)
    test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)
    test_one_hot_predict_series_batch = list(test_one_hot_predict_df.idxmax(axis = 1))
    test_one_hot_predict_series = test_one_hot_predict_series + test_one_hot_predict_series_batch 
    
    test_one_hot_truth = (sg_test_no_wavelet[ii][1] > 0.9).astype(int)
    test_one_hot_truth_df = pd.DataFrame(test_one_hot_truth,columns=label_names_df)
    test_one_hot_truth_series_batch = list(test_one_hot_truth_df.idxmax(axis = 1))
    test_one_hot_truth_series = test_one_hot_truth_series + test_one_hot_truth_series_batch

In [119]:
ax = plt.subplot(2,1,1)

plotResult_colorbars(test_one_hot_predict_series, range(len(test_one_hot_predict_series)),
                     labelNames=label_types,ax=ax,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_predict_series)])
ax = plt.subplot(2,1,2)

plotResult_colorbars(test_one_hot_truth_series, range(len(test_one_hot_truth_series)),
                     labelNames=label_types,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_truth_series)])

(0, 1900)